# 🔍 Bitcoin TDA Real-time Monitor Dashboard

**Topological Data Analysis** 기반 비트코인 극단 이벤트 탐지 대시보드

---

In [ ]:
# Import libraries
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
from IPython.display import display, HTML, clear_output
import json
from datetime import datetime
import time

# Import TDA monitor
from monitor import TDAMonitor

print("✅ 라이브러리 로드 완료!")

## 📊 L1 Norm 이해하기

### 🎯 핵심 개념

**L1 Norm = 시장의 위상학적 복잡도**

- **낮은 값 (< 70%)**: 🏞️ 평온한 시장 - 안정적
- **중간 값 (70-90%)**: 🌊 변동성 증가 - 주의 필요
- **높은 값 (> 100%)**: 🌪️ 극단 이벤트 - 위험!

### 📈 실제 의미

```
L1 Norm이 높다 = 시장 구조가 복잡하다 = 변동성이 크다
```

---

In [ ]:
# Initialize Monitor
print("🔄 TDA Monitor 초기화 중...")
print("(최초 실행 시 1-2분 소요될 수 있습니다)")

monitor = TDAMonitor(
    historical_data_path='/notebooks/binance-data-collector/BTCUSDT_5m.csv',
    window_size=60,
    lookback_days=30
)

print("\n✅ Monitor 초기화 완료!")

---

## 🎯 현재 시장 상태

### 실시간 분석 결과

In [ ]:
# Run analysis
print("📊 현재 시장 상태 분석 중...")
analysis = monitor.run_once()

if analysis:
    # Extract data
    timestamp = analysis['timestamp']
    price = analysis['price']
    l1_val = analysis['metrics']['l1_norm']['value']
    l1_pct = analysis['metrics']['l1_norm']['percent_of_threshold']
    l1_thr = analysis['metrics']['l1_norm']['threshold']
    l2_val = analysis['metrics']['l2_norm']['value']
    l2_pct = analysis['metrics']['l2_norm']['percent_of_threshold']
    l2_thr = analysis['metrics']['l2_norm']['threshold']
    alert = analysis['alert_level']
    
    # Display Alert Banner
    alert_colors = {
        'NORMAL': '#4caf50',
        'WARNING': '#ffeb3b',
        'SEVERE': '#ff9800',
        'CRITICAL': '#f44336'
    }
    
    alert_html = f"""
    <div style="background-color: {alert_colors[alert['level']]}; 
                padding: 20px; 
                border-radius: 10px; 
                margin: 10px 0;
                border-left: 8px solid {'#2e7d32' if alert['level']=='NORMAL' else '#c62828'};">
        <h2 style="margin: 0; color: white;">{alert['symbol']} {alert['level']}</h2>
        <p style="margin: 10px 0 0 0; color: white; font-size: 16px;">{alert['message']}</p>
    </div>
    """
    display(HTML(alert_html))
    
    # Display metrics
    metrics_html = f"""
    <div style="display: grid; grid-template-columns: repeat(4, 1fr); gap: 15px; margin: 20px 0;">
        <div style="background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); 
                    padding: 20px; border-radius: 10px; color: white; text-align: center;">
            <div style="font-size: 14px; opacity: 0.9;">💰 현재 가격</div>
            <div style="font-size: 28px; font-weight: bold; margin-top: 10px;">${price:,.2f}</div>
        </div>
        <div style="background: linear-gradient(135deg, #f093fb 0%, #f5576c 100%); 
                    padding: 20px; border-radius: 10px; color: white; text-align: center;">
            <div style="font-size: 14px; opacity: 0.9;">📊 L¹ Norm</div>
            <div style="font-size: 28px; font-weight: bold; margin-top: 10px;">{l1_val:.4f}</div>
            <div style="font-size: 14px; margin-top: 5px;">{l1_pct:.1f}% of threshold</div>
        </div>
        <div style="background: linear-gradient(135deg, #4facfe 0%, #00f2fe 100%); 
                    padding: 20px; border-radius: 10px; color: white; text-align: center;">
            <div style="font-size: 14px; opacity: 0.9;">📈 L² Norm</div>
            <div style="font-size: 28px; font-weight: bold; margin-top: 10px;">{l2_val:.4f}</div>
            <div style="font-size: 14px; margin-top: 5px;">{l2_pct:.1f}% of threshold</div>
        </div>
        <div style="background: linear-gradient(135deg, #43e97b 0%, #38f9d7 100%); 
                    padding: 20px; border-radius: 10px; color: white; text-align: center;">
            <div style="font-size: 14px; opacity: 0.9;">⏰ 업데이트</div>
            <div style="font-size: 20px; font-weight: bold; margin-top: 10px;">{timestamp.strftime('%H:%M:%S')}</div>
            <div style="font-size: 14px; margin-top: 5px;">{timestamp.strftime('%Y-%m-%d')}</div>
        </div>
    </div>
    """
    display(HTML(metrics_html))
    
    print("\n✅ 분석 완료!")
else:
    print("❌ 분석 실패")

---

## 📊 게이지 차트 - 현재 상태

In [ ]:
# Create Gauge Charts
def create_gauge(value, threshold, title, color='blue'):
    percent = (value / threshold) * 100
    
    fig = go.Figure(go.Indicator(
        mode="gauge+number+delta",
        value=percent,
        domain={'x': [0, 1], 'y': [0, 1]},
        title={'text': title, 'font': {'size': 24, 'color': '#333'}},
        delta={'reference': 100, 'suffix': '%'},
        number={'suffix': '%', 'font': {'size': 40}},
        gauge={
            'axis': {'range': [None, 150], 'tickwidth': 2, 'tickcolor': "darkblue"},
            'bar': {'color': color, 'thickness': 0.8},
            'bgcolor': "white",
            'borderwidth': 3,
            'bordercolor': "gray",
            'steps': [
                {'range': [0, 70], 'color': '#e8f5e9'},
                {'range': [70, 90], 'color': '#fffde7'},
                {'range': [90, 100], 'color': '#fff3e0'},
                {'range': [100, 150], 'color': '#ffebee'}
            ],
            'threshold': {
                'line': {'color': "red", 'width': 6},
                'thickness': 0.85,
                'value': 100
            }
        }
    ))
    
    fig.update_layout(
        height=350,
        margin=dict(l=40, r=40, t=80, b=40),
        font={'family': 'Arial, sans-serif'}
    )
    
    return fig

# L1 Norm Gauge
fig_l1 = create_gauge(l1_val, l1_thr, "L¹ Norm Status", color='#f59e0b')
fig_l1.show()

# L2 Norm Gauge
fig_l2 = create_gauge(l2_val, l2_thr, "L² Norm Status", color='#10b981')
fig_l2.show()

---

## 📈 시계열 분석 - 과거 추이

In [ ]:
# Create time series plot
df = monitor.df_history.tail(1000).copy()

# Prepare data for TDA
features = {
    'close': df['close'].values,
    'volume': df['volume'].values,
    'volume_delta': df['volume_delta'].values,
    'cvd': df['cvd'].values
}

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
for key in features:
    features[key] = scaler.fit_transform(features[key].reshape(-1, 1)).flatten()

point_cloud = monitor.detector.create_multivariate_point_cloud(features)
results = monitor.detector.sliding_window_analysis(point_cloud, homology_dim=1)

# Create subplot figure
fig = make_subplots(
    rows=3, cols=1,
    shared_xaxes=True,
    vertical_spacing=0.05,
    subplot_titles=('Bitcoin Price (USDT)', 'L¹ Norm', 'L² Norm'),
    row_heights=[0.4, 0.3, 0.3]
)

timestamps = df['timestamp'].values
plot_timestamps = timestamps[monitor.window_size-1:][:len(results['l1_norms'])]

# Price chart
fig.add_trace(
    go.Scatter(
        x=timestamps,
        y=df['close'].values,
        mode='lines',
        name='Price',
        line=dict(color='#667eea', width=2),
        fill='tozeroy',
        fillcolor='rgba(102, 126, 234, 0.1)'
    ),
    row=1, col=1
)

# L1 Norm
l1_threshold = monitor.baseline_stats['l1_norm']['threshold']
fig.add_trace(
    go.Scatter(
        x=plot_timestamps,
        y=results['l1_norms'],
        mode='lines',
        name='L¹ Norm',
        line=dict(color='#f59e0b', width=2)
    ),
    row=2, col=1
)
fig.add_hline(
    y=l1_threshold,
    line_dash="dash",
    line_color="red",
    line_width=2,
    annotation_text="Threshold (μ + 4σ)",
    annotation_position="right",
    row=2, col=1
)

# L2 Norm
l2_threshold = monitor.baseline_stats['l2_norm']['threshold']
fig.add_trace(
    go.Scatter(
        x=plot_timestamps,
        y=results['l2_norms'],
        mode='lines',
        name='L² Norm',
        line=dict(color='#10b981', width=2)
    ),
    row=3, col=1
)
fig.add_hline(
    y=l2_threshold,
    line_dash="dash",
    line_color="red",
    line_width=2,
    annotation_text="Threshold (μ + 4σ)",
    annotation_position="right",
    row=3, col=1
)

fig.update_layout(
    height=900,
    showlegend=True,
    hovermode='x unified',
    title_text="📊 Bitcoin TDA Time Series Analysis",
    title_font_size=24
)

fig.update_xaxes(title_text="Time", row=3, col=1)
fig.update_yaxes(title_text="Price (USDT)", row=1, col=1)
fig.update_yaxes(title_text="L¹ Norm", row=2, col=1)
fig.update_yaxes(title_text="L² Norm", row=3, col=1)

fig.show()

---

## 🔬 Persistence Diagram - 위상학적 특징

In [ ]:
# Plot Persistence Diagram
dgm = analysis['persistence_diagram']
dgm = dgm[dgm[:, 1] < np.inf]  # Remove infinite points

if len(dgm) > 0:
    fig = go.Figure()
    
    # Add points
    persistence = dgm[:, 1] - dgm[:, 0]
    fig.add_trace(go.Scatter(
        x=dgm[:, 0],
        y=dgm[:, 1],
        mode='markers',
        marker=dict(
            size=10,
            color=persistence,
            colorscale='Viridis',
            showscale=True,
            colorbar=dict(title="Persistence", thickness=20)
        ),
        name='H₁ features',
        text=[f'Birth: {b:.4f}<br>Death: {d:.4f}<br>Persistence: {d-b:.4f}'
              for b, d in dgm],
        hovertemplate='%{text}<extra></extra>'
    ))
    
    # Add diagonal
    max_val = max(dgm[:, 1].max(), dgm[:, 0].max())
    fig.add_trace(go.Scatter(
        x=[0, max_val],
        y=[0, max_val],
        mode='lines',
        line=dict(dash='dash', color='red', width=2),
        name='Diagonal',
        showlegend=True
    ))
    
    fig.update_layout(
        title='🔬 Persistence Diagram (H₁ - 1차원 Homology)',
        xaxis_title='Birth',
        yaxis_title='Death',
        height=500,
        hovermode='closest',
        title_font_size=20
    )
    
    fig.show()
    
    print(f"\n📊 총 {len(dgm)}개의 H₁ 특징 발견")
    print(f"평균 Persistence: {persistence.mean():.4f}")
    print(f"최대 Persistence: {persistence.max():.4f}")
else:
    print("현재 윈도우에서 persistent features가 발견되지 않았습니다.")

---

## 📋 Baseline Statistics

In [ ]:
# Display baseline statistics table
stats_data = {
    'Metric': ['L¹ Norm', 'L² Norm', 'Wasserstein Distance'],
    'Mean (μ)': [
        monitor.baseline_stats['l1_norm']['mean'],
        monitor.baseline_stats['l2_norm']['mean'],
        monitor.baseline_stats['wasserstein']['mean']
    ],
    'Std (σ)': [
        monitor.baseline_stats['l1_norm']['std'],
        monitor.baseline_stats['l2_norm']['std'],
        monitor.baseline_stats['wasserstein']['std']
    ],
    'Threshold (μ+4σ)': [
        monitor.baseline_stats['l1_norm']['threshold'],
        monitor.baseline_stats['l2_norm']['threshold'],
        monitor.baseline_stats['wasserstein']['threshold']
    ]
}

stats_df = pd.DataFrame(stats_data)

display(HTML("<h3>📊 Baseline Statistics (최근 30일 기준)</h3>"))
display(stats_df.style.format({
    'Mean (μ)': '{:.6f}',
    'Std (σ)': '{:.6f}',
    'Threshold (μ+4σ)': '{:.6f}'
}).background_gradient(cmap='YlOrRd', subset=['Threshold (μ+4σ)']))

---

## 🔄 자동 새로고침 (Optional)

아래 셀을 실행하면 5분마다 자동으로 업데이트됩니다. (Ctrl+C로 중단)

In [ ]:
# Auto-refresh (실행하면 계속 반복)
try:
    interval = 300  # 5분 (300초)
    
    while True:
        clear_output(wait=True)
        
        print("🔄 새로고침 중...")
        
        # Re-run analysis
        analysis = monitor.run_once()
        
        if analysis:
            timestamp = analysis['timestamp']
            price = analysis['price']
            l1_val = analysis['metrics']['l1_norm']['value']
            l1_pct = analysis['metrics']['l1_norm']['percent_of_threshold']
            alert = analysis['alert_level']
            
            print(f"\n{'='*70}")
            print(f"🔍 BITCOIN TDA MONITOR - {timestamp}")
            print(f"{'='*70}")
            print(f"\n💰 Current Price: ${price:,.2f}")
            print(f"\n📊 L¹ Norm: {l1_val:.4f} ({l1_pct:.1f}% of threshold)")
            print(f"\n🎯 STATUS: {alert['symbol']} {alert['level']}")
            print(f"   {alert['message']}")
            print(f"\n{'='*70}")
        
        print(f"\n⏱️  다음 업데이트: {interval}초 후...")
        print("(Ctrl+C로 중단)")
        
        time.sleep(interval)
        
except KeyboardInterrupt:
    print("\n✋ 자동 새로고침 중단됨")

---

## 💡 사용 팁

### L1 Norm 해석:
- **< 70% (녹색)**: ✅ 정상 - 안정적 시장
- **70-90% (노란색)**: ⚡ 주의 - 변동성 증가
- **90-100% (주황색)**: ⚠️ 경고 - 극단 이벤트 임박
- **> 100% (빨간색)**: 🚨 극단 - 극단 이벤트 발생!

### 트레이딩 전략:
1. **70% 도달**: 포지션 크기 줄이기
2. **90% 도달**: Stop-loss 강화
3. **100% 초과**: 신규 진입 자제

---

**Paper**: [arXiv:2405.16052](https://arxiv.org/abs/2405.16052)